# scqubits example: the Explorer class
J. Koch and P. Groszkowski

For further documentation of scqubits see https://scqubits.readthedocs.io/en/latest/.

---

# The Explorer Class

Sometimes, exploring the properties of coupled quantum systems benefits from visual aides, and the possibility to see how properties change when system parameters are changed. The `Explorer` class in scqubits provides an interactive plot with multiple panels. 

The idea behind the `Explorer` class is simple: the user selects an external parameter that they wish to sweep, e.g., an external magnetic flux. The composite system is user-defined through the `HilbertSpace` and `InteractionTerm` classes, allowing flexibility to include different types and numbers of superconducting qubits and harmonic modes.

Once defined, the parameter sweep is computed by means of the `ParameterSweep` class, and spectral data is stored in memory to allow efficient, interactive display of data. The `Explorer` supports plots of:

1. Bare spectra of the individual qubits
2. Wave functions of the bare qubits
3. Dressed spectrum of the composite Hilbert space
4. Spectrum for n-photon qubit transitions, starting from a given initial state
5. AC Stark shift $\chi_{01}$ for any of the qubits
6. Charge matrix elements for any of the qubits, using the same initial state as in point 4.

## Example 1: fluxonium coupled to resonator
As a first example, we consider a system composed of a fluxonium qubit, coupled through its charge operator to the voltage inside a resonator.

In [1]:
import numpy as np
import scqubits as scq

qbt = scq.Fluxonium(
    EJ=2.55,
    EC=0.72,
    EL=0.12,
    flux=0.0,
    cutoff=110,
    truncated_dim=9
)

osc = scq.Oscillator(
    E_osc=4.0,
    truncated_dim=5
)

hilbertspace = scq.HilbertSpace([qbt, osc])

hilbertspace.add_interaction(
    g_strength=0.2,
    op1=qbt.n_operator,
    op2=osc.creation_operator,
    add_hc=True
)

param_name = 'Φext/Φ0'
param_vals = np.linspace(-0.5, 0.5, 101)

subsys_update_list = [qbt]


def update_hilbertspace(param_val):
    qbt.flux = param_val


sweep = scq.ParameterSweep(
    paramvals_by_name={param_name: param_vals},
    evals_count=20,
    hilbertspace=hilbertspace,
    subsys_update_info={param_name: [qbt]},
    update_hilbertspace=update_hilbertspace,
)

Bare spectra:   0%|          | 0/101 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/1 [00:00<?, ?it/s]

Dressed spectrum:   0%|          | 0/101 [00:00<?, ?it/s]

### Starting the Explorer class
At this point, everything is prepared to start the interactive `Explorer` display!

In [2]:
xp = scq.Explorer(sweep)

Container(children=[Sheet(children=[Card(children=[Img(layout=None, src='data:image/png;base64,iVBORw0KGgoAAAA…

## Example 2:  two transmons coupled to a resonator
In the following second example, we consider a system composed of two flux-tunable transmons, capacitively coupled to one joint harmonic mode. (The flux is assumed to arise from a global field, and the SQUID-loop areas of the two transmons are different with an area ratio of 1.4) 

### Hilbert space setup

In [3]:
qbt1 = scq.Transmon(
     EJ=25.0,
     EC=0.2,
     ng=0,
     ncut=30,
     truncated_dim=3)

qbt2 = scq.Transmon(
     EJ=15.0,
     EC=0.15,
     ng=0,
     ncut=30,
     truncated_dim=3)


resonator = scq.Oscillator(
    E_osc=5.5,
    truncated_dim=4)

hilbertspace = scq.HilbertSpace([qbt1, qbt2, resonator])


g1 = 0.1  # coupling resonator-CPB1 (without charge matrix elements)
g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)

hilbertspace.add_interaction(
    g_strength = g1,
    op1 = qbt1.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True
)

hilbertspace.add_interaction(
    g_strength = g2,
    op1 = qbt2.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True
)

### Parameter sweep setup

In [4]:
param_name = r'$\Phi_{ext}/\Phi_0$'
param_vals = np.linspace(0.0, 1.0, 151)

subsys_update_list = [qbt1, qbt2]


def update_hilbertspace(param_val):     # function that shows how Hilbert space components are updated
    qbt1.EJ = 30*np.abs(np.cos(np.pi * param_val))
    qbt2.EJ = 40*np.abs(np.cos(np.pi * param_val * 2))


sweep = scq.ParameterSweep(
    paramvals_by_name={param_name: param_vals},
    evals_count=15,
    hilbertspace=hilbertspace,
    subsys_update_info={param_name: [qbt1, qbt2]},
    update_hilbertspace=update_hilbertspace,
)

Bare spectra:   0%|          | 0/151 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/151 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/1 [00:00<?, ?it/s]

Dressed spectrum:   0%|          | 0/151 [00:00<?, ?it/s]

### Start Explorer

In [5]:
explorer = scq.Explorer(sweep=sweep)

Container(children=[Sheet(children=[Card(children=[Img(layout=None, src='data:image/png;base64,iVBORw0KGgoAAAA…